# 언어 모델을 사용한 문장 생성

기계 번역, 음성 인식, 문장 생성

## RNN을 사용한 문장 생성의 순서

<center> you say goodbye and "I" say hello.    </center> 
<center>    *** I에 대한 확률분포 *** </center>
    
<img src = "../imgs/fig 7-2.png" width = "200">
    
#### HOW TO 생성, I 다음의 단어?

1. 확률이 가장 높은 단어를 선택한다 
    - Deterministic 한 결과
    - 결과가 바뀌지 않음
    - `say` 라는 단어 선택됨. 고정 불변.

2. 결과를 확률에 따라 다르게
    - 높은 단어가 선택되기 쉽고 낮은 단어는 선택되기 어려워짐
    - `say` 라는 단어가 선택되기 쉬워짐.

<img src = "../imgs/fig 7-4.png" width = "400">

- 반복..반복.. until 원하는 만큼 혹은 `<eos>` 와 같은 종결 기호가 나올 때까지


## 문장 생성 구현

In [7]:
import sys 
sys.path.append("..")
import numpy as np
from common.functions import softmax
from ch06_게이트가_추가된_RNN.rnnlm import Rnnlm
from ch06_게이트가_추가된_RNN.better_rnnlm import BetterRnnlm

class RnnlmGen(Rnnlm):
    # 문장 생성 수행. 100 단어까지
    def generate(self, start_id, skip_ids=None, sample_size=100):
        ''' start_id : 최초로 주는 단어의 ID
            skip_ids : 해당 리스트에 속하는 단어 ID는 샘플링 되지 않도록 방지. (전처리된 단어 등)
        '''
        word_ids = [start_id]
        
        x = start_id
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(-1,1) # 미니배치 처리때문에 x는 2차원. 1X1로 변형.
            score = self.predict(x) # 각 단어의 점수 출력
            p = softmax(score.flatten()) # softmax 함수 통해 정규화하여 확률분포 p 얻기

            sampled = np.random.choice(len(p),size=1, p=p) # p로부터 다음 단어 샘플링

            if (skip_ids is None) or (sampled not in skip_ids):
                x = sampled
                word_ids.append(int(x))
                
        return word_ids
    
    def get_state(self):
        return self.lstm_layer.h, self.lstm_layer.c

    def set_state(self, state):
        self.lstm_layer.set_state(*state)

In [57]:
import sys 
sys.path.append("..")
from rnnlm_gen import RnnlmGen
from dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

model = RnnlmGen()
model.load_params('../ch06_게이트가_추가된_RNN/Rnnlm.pkl') # 미리 학습된 가중치 불러와 성능 높임

# 시작 (start) 문자와 건너뛸 (skip) 문자 설정
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# 문장 생성
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace('<eos>', '.\n')
print(txt)

you impression shared georgia nonsense bleeding devastation department-store styles ratios receiving long-distance fell phoenix raiders mateo performers enough matter feel pitches stem sentence routinely commons abortion-rights legislators iverson inaccurate forth assurance stearns va supposedly tharp mlx spree spots boren grow separately virus peck carol pitch reject something sound proving one-day u priced cargo mandate laurel we equivalent requested shipment focused franco r.h. enjoys forced proceedings method having sohmer budgetary achievement processors genuine taped startling predicted fool aging england denounced liquidated taipei mason graham photograph fe bougainville faced revise nbc refinancing until beach pride strange vickers engine sample ambitious entrenched d'arcy


그러나 별로 성능이 좋진 않음..

## 더 좋은 문장으로

In [63]:
class BetterRnnlmGen(BetterRnnlm):
    def generate(self, start_id, skip_ids=None, sample_size=100):
        word_ids = [start_id]

        x = start_id
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(1, 1)
            score = self.predict(x).flatten()
            p = softmax(score).flatten()

            sampled = np.random.choice(len(p), size=1, p=p)
            if (skip_ids is None) or (sampled not in skip_ids):
                x = sampled
                word_ids.append(int(x))

        return word_ids

    def get_state(self):
        states = []
        for layer in self.lstm_layers:
            states.append((layer.h, layer.c))
        return states

    def set_state(self, states):
        for layer, state in zip(self.lstm_layers, states):
            layer.set_state(*state)

In [7]:
import sys 
sys.path.append("..")
import numpy as np
from rnnlm_gen import BetterRnnlmGen
from dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

model = BetterRnnlmGen()
model.load_params('../ch06_게이트가_추가된_RNN/BetterRnnlm.pkl') # 미리 학습된 가중치 불러와 성능 높임

# 시작 (start) 문자와 건너뛸 (skip) 문자 설정
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# 문장 생성
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace('<eos>', '.\n')
print(txt)

you are going to think mr. noble says .
 nothing will be on the group .
 that 's not that the business investment service does n't hold a great consideration .
 it 's only tough recently for all one devoted to the petroleum and economic services and assure good stuff .
 mr. gould says he will pay the reliance stake in a unit that is sold in associates research for the mgm grand foundation .
 and on london 's stock market reports mr. johnson notes that the diversified real estate industry is n't the only consideration that it 's not


오 결과 좋아 

### "the meaning of life is" 다음 이어지는 문장 생성하기

- `the` `meaning` `of` `life` 단어를 차례로 주어 순전파 수행
- 출력 결과는 무시하고 단어열 정보를 유지
- 그 다음 `is`를 첫 단어로 입력해 문장 생성 시작

In [22]:
model.reset_state() # not continue the previous sequences anymore, now you will start feeding new sequences.

start_words = 'the meaning of life is'
start_ids = [word_to_id[w] for w in start_words.split(' ')]

for x in start_ids[:-1]:
    x = np.array(x).reshape(1, 1)
    model.predict(x) # shape = (1, 1, 10000)
    
word_ids = model.generate(start_ids[-1], skip_ids)
word_ids = start_ids[:-1] + word_ids
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print('-' * 50)
print(txt)

--------------------------------------------------
the meaning of life is selling in texas and there are a few well-known reflected yet at least three days.
 american donations of the south korea and wisconsin.
 in an apparent letter to dpc acquisition corp. said it is trying to repurchase two competing partners as the bank of london launched a friendly pact with american medical corp. a unit of a food concern.
 the move reflected a temporary charge in japan 's real estate business.
 and analysts caution the short-term financing moves traditionally weaker some great debt will delay market share of program trading.
 it seems to make


# seq2seq

- 시계열 데이터를 또 다른 시계열 데이터로 변환하는 문제
    - 기계 번역 (언어 -> 언어)
    - 음성 인식 (음성 -> 언어)
    - 챗봇 어플리케이션 (대화 -> 대화)
    - 컴파일러 (소스코드 -> 기계어)
 

==> **시계열 데이터를 다른 시계열 데이터로 변환하는 모델**
  
  - 2 개의 RNN을 이용하는 **seq2seq <sup>sequence to sequence</sup>**

## seq2seq의 원리

- seq2seq는 **Encoder-Decoder 모델**이라고도 한다
    - `Encoder` : 입력 데이터를 인코딩 (부호화)
    - `Decoder` : 인코딩된 데이터를 디코딩 (복호화)
    
    - <img src = "../imgs/fig 7-5.png" width = "200">

---
#### Encoder

<img src = "../imgs/fig 7-6.png" width = "700">

- RNN(LSTM)을 이용해 시계열 데이터를 **h**라는 은닉 상태 벡터로 변환
- 문장을 단어 단위로 쪼개서 입력한다

- ### `중요한 점` **LSTM의 은닉 상태 h는 고정 길이 벡터라는 사실**
- 즉, 인코딩은 임의 길이의 문장을 고정 길이 벡터로 변환하는 작업임

---
#### Decoder

<img src = "../imgs/fig 7-8.png" width = "700">

- 앞 절에서 다룬 문장 생성 모델!
- **LSTM 계층이 벡터 h를 입력받는다는 점이 차이**. 
- 앞 절의 언어 모델에서는 LSTM 계층이 아무것도 받지 않음 ㅎㅎ


##### c.f.

- `<eos>` : 문장 생성의 시작을 알리는 신호이자 종료 신호. 구분자. 
- `<go>` `<start>` `_`

#### seq2seq : Encoder + Decoder
<img src = "../imgs/fig 7-9.png" width = "700">

- LSTM 두 개

    - Encoder의 LSTM + Decoder의 LSTM 
    - 둘은 은닉 상태 h로 이어진다
    
    
- 순전파 : Encoder -> h -> Decoder
- 역전파 : Decoder -> h -> Encoder

## 시계열 데이터 변환용 Toy Problem

<img src = "../imgs/fig 7-10.png" width = "300">

#### <center>57 + 5 ==> [5, 7, + 5]</center>

## 가변 길이 시계열 데이터

- **덧셈 문제에서는 샘플마다 데이터의 시간 방향 크기가 다르다!**
- `57+5`는 4 문자, `628+123`은 7 문자
- 가변 길이의 시계열 데이터

신경망에 미니배치 처리를 하기 위한 가장 심플한 방법

- **패딩 (padding)**
    - 원래의 데이터에 의미 없는 데이터를 채워 모든 데이터 길이 동일하게 맞추기

<img src = "../imgs/fig 7-11.png" width = "500">

- 문제
    - 덧셈 문제 : 0-999 사이. 최대 7 글자
    - 정답 : 최대 999+999=1998. 최대 4글자
    - 구분자 : `_`
- 인풋 아웃풋
    - 입력 : 7글자
    - 출력 : 정답 + 구분자 = 5글자
    
---

- 원래는 존재하지 않던 패딩용 문자까지 모델이 처리하므로 정확성 떨어짐
- 정확성을 올리려면 추가로 패딩 전용 처리를 해줘야 함
    - Decoder에 입력된 데이터가 패딩이라면 손실의 결과에 반영하지 않도록.
        - ( Softmax with Loss + mask )
    - Encdoer에 입력된 데이터가 패딩이라면 LSTM 계층이 이전 시각의 입력을 그대로 출력
        - 원래부터 패딩은 없었어..!